In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# ==============================================================================
# 1. CONFIGURACIÓN: RUTAS DE TUS ARCHIVOS LOCALES
# ==============================================================================
# EDITA ESTO CON LOS NOMBRES REALES DE TUS ARCHIVOS
rutas_archivos = {
    "combustible": "Enero-febrero 2025 Combustible.xlsx",  # Cambia por tu nombre real
    "viajes": "Reporte Listado Viajes Ene-Oct_25.xlsx", # Cambia por tu nombre real
    "telemetria": "Telemetría.xlsx",            # Cambia por tu nombre real
    "tablero": "Tablero_PL_2024-09_2025-09_v2.xlsx" # Cambia por tu nombre real
}

print("--- INICIANDO PROCESAMIENTO DE DATOS ---")

# Diccionario donde guardaremos todo
cache_data = {}

# ------------------------------------------------------------------------------
# 2. PROCESAR COMBUSTIBLE
# ------------------------------------------------------------------------------
try:
    print(f"Procesando Combustible: {rutas_archivos['combustible']}...")
    df = pd.read_excel(rutas_archivos["combustible"])
    if 'Precio Unitario Merc' in df.columns:
        # Limpieza
        df['Precio Unitario Merc'] = pd.to_numeric(
            df['Precio Unitario Merc'].astype(str).str.replace(r'[$,]', '', regex=True), 
            errors='coerce'
        )
        val = df[df['Precio Unitario Merc'] > 0]['Precio Unitario Merc'].mean()
        cache_data['combustible_price'] = val
        print(f"   -> Precio promedio calculado: ${val:.2f}")
except Exception as e:
    print(f"   [ERROR] Combustible: {e}")
    cache_data['combustible_price'] = 0

# ------------------------------------------------------------------------------
# 3. PROCESAR VIAJES (RUTAS + CLUSTER + DATA RAW)
# ------------------------------------------------------------------------------
try:
    print(f"Procesando Viajes: {rutas_archivos['viajes']}...")
    df = pd.read_excel(rutas_archivos["viajes"])
    
    # Asegurar columna Viaje
    if 'Viaje' not in df.columns: df['Viaje'] = 1
    
    # Filtro de columnas para reducir peso
    cols_needed = ['Ruta', 'Fecha Salida', 'Fecha Llegada', 'Estatus de Viaje', 
                   'Nro Ope', 'Operador', 'Tractocamión', 'Viaje', 'Nombre Cliente']
    cols_present = [c for c in cols_needed if c in df.columns]
    df = df[cols_present]
    df = df.drop_duplicates()
    
    # A) Rutas Options
    rutas_unicas = df['Ruta'].dropna().unique()
    cache_data['rutas_options'] = [{'label': str(r), 'value': str(r)} for r in rutas_unicas]
    
    # B) Guardar Data Procesada (JSON format para Dash)
    # Convertimos fechas a string para serialización, luego Dash las recarga
    df['Fecha Salida'] = df['Fecha Salida'].astype(str)
    df['Fecha Llegada'] = df['Fecha Llegada'].astype(str)
    cache_data['viajes_processed'] = df.to_dict('records')
    
    # C) Clustering (Modelo IA)
    # Reconvertimos a datetime temporalmente para el cálculo
    df['Fecha Salida'] = pd.to_datetime(df['Fecha Salida'], errors='coerce')
    df['Fecha Llegada'] = pd.to_datetime(df['Fecha Llegada'], errors='coerce')
    
    df_model = df.dropna(subset=['Fecha Salida', 'Estatus de Viaje']).copy()
    mask = (df_model['Fecha Salida'].dt.year >= 2024)
    df_model = df_model[mask]

    ruta_volumen = df_model.groupby('Ruta')['Viaje'].count().reset_index(name='Total_Viajes')
    ruta_cliente = df_model.groupby(['Ruta', 'Nombre Cliente'])['Viaje'].count().reset_index()
    ruta_max_cliente = ruta_cliente.sort_values(['Ruta', 'Viaje'], ascending=[True, False]).groupby('Ruta').first().reset_index()
    
    data_cluster = pd.merge(ruta_volumen, ruta_max_cliente[['Ruta', 'Viaje']], on='Ruta')
    data_cluster['Porcentaje_Dependencia'] = data_cluster['Viaje'] / data_cluster['Total_Viajes']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data_cluster[['Total_Viajes', 'Porcentaje_Dependencia']])
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    data_cluster['Cluster_Riesgo'] = kmeans.fit_predict(X_scaled)

    # Mapeo Inteligente
    analysis = data_cluster.groupby('Cluster_Riesgo')['Porcentaje_Dependencia'].mean().sort_values(ascending=False)
    id_critico = analysis.index[0] 
    id_estable = analysis.index[-1]
    id_moderado = [x for x in [0,1,2] if x not in [id_critico, id_estable]][0]
    
    mapa = {int(id_critico): "CRÍTICO", int(id_moderado): "MODERADO", int(id_estable): "ESTABLE"}
    
    res_cluster = {}
    for _, row in data_cluster.iterrows():
        res_cluster[row['Ruta']] = mapa.get(int(row['Cluster_Riesgo']), "DESCONOCIDO")
        
    cache_data['cluster_results'] = res_cluster
    print(f"   -> Clustering completado. {len(res_cluster)} rutas clasificadas.")

except Exception as e:
    print(f"   [ERROR] Viajes: {e}")
    cache_data['rutas_options'] = []
    cache_data['viajes_processed'] = None
    cache_data['cluster_results'] = None

# ------------------------------------------------------------------------------
# 4. PROCESAR FINANCIEROS (TELEMETRIA + TABLERO)
# ------------------------------------------------------------------------------
try:
    print("Procesando Financieros (Telemetría + PL)...")
    
    # Telemetría
    try: df_tele = pd.read_excel(rutas_archivos["telemetria"], sheet_name="Report")
    except: df_tele = pd.read_excel(rutas_archivos["telemetria"])
    
    cols_tele = ['Nombre', 'Periodo', 'Distancia', 'Fecha de inicio del periodo', 'Fecha de fin del periodo']
    cols_ok_tele = [c for c in cols_tele if c in df_tele.columns]
    
    # Convertir fechas a str para guardar
    df_tele['Fecha de inicio del periodo'] = df_tele['Fecha de inicio del periodo'].astype(str)
    df_tele['Fecha de fin del periodo'] = df_tele['Fecha de fin del periodo'].astype(str)
    
    cache_data['telemetria_data'] = df_tele[cols_ok_tele].to_dict('records')
    
    # Tablero PL
    try: df_pl = pd.read_excel(rutas_archivos["tablero"], sheet_name="Long PL Meses")
    except: df_pl = pd.read_excel(rutas_archivos["tablero"])
    
    df_pl['Date'] = df_pl['Date'].astype(str)
    cache_data['tablero_data'] = df_pl[['Rubro', 'Date', 'Real']].to_dict('records')

    # --- CÁLCULO PREVIO DE KPIs ---
    # Reconstruimos DFs para calcular CPK y Ociosidad aquí mismo y guardarlos ya calculados
    df_tele_calc = pd.DataFrame(cache_data['telemetria_data'])
    df_pl_calc = pd.DataFrame(cache_data['tablero_data'])
    
    df_tele_calc['Fecha de inicio del periodo'] = pd.to_datetime(df_tele_calc['Fecha de inicio del periodo'])
    df_tele_calc['Mes'] = df_tele_calc['Fecha de inicio del periodo'].dt.to_period('M').astype(str)
    
    df_pl_calc['Date'] = pd.to_datetime(df_pl_calc['Date'])
    df_pl_calc['Mes'] = df_pl_calc['Date'].dt.to_period('M').astype(str)

    # CPK
    dist_mes = df_tele_calc.groupby('Mes')['Distancia'].sum().reset_index()
    cost_mes = df_pl_calc[df_pl_calc["Rubro"] == "2. Costo"][["Mes", "Real"]]
    merged = pd.merge(cost_mes, dist_mes, on="Mes")
    cpk = (merged["Real"] / merged["Distancia"]).mean()
    cache_data['cpk_final'] = cpk

    # Ociosidad
    unidades = len(df_tele_calc["Nombre"].unique())
    ultimo_mes = df_pl_calc['Date'].max()
    rubros = ['Salarios', 'Mantenimiento Automotriz', 'Depreciación', 'Arrendamiento']
    costo_fijo = df_pl_calc[(df_pl_calc['Rubro'].isin(rubros)) & (df_pl_calc['Date'] == ultimo_mes)]['Real'].sum()
    
    ociosidad = (costo_fijo / unidades / 30) if unidades > 0 else 0
    cache_data['ociosidad_global'] = ociosidad
    
    print(f"   -> Financieros OK. CPK: ${cpk:.2f}, Ociosidad: ${ociosidad:.2f}/día")

except Exception as e:
    print(f"   [ERROR] Financieros: {e}")
    cache_data['telemetria_data'] = None
    cache_data['tablero_data'] = None
    cache_data['cpk_final'] = 0
    cache_data['ociosidad_global'] = 0

# ------------------------------------------------------------------------------
# 5. GUARDAR ARCHIVO PKL
# ------------------------------------------------------------------------------
nombre_archivo = "datos_dashboard_v12.pkl"
with open(nombre_archivo, "wb") as f:
    pickle.dump(cache_data, f)

print(f"\n✅ ¡ÉXITO! Archivo '{nombre_archivo}' generado correctamente.")
print("Ahora ejecuta 'app_logistica_final_v12.py'.")

--- INICIANDO PROCESAMIENTO DE DATOS ---
Procesando Combustible: Enero-febrero 2025 Combustible.xlsx...
   -> Precio promedio calculado: $24.95
Procesando Viajes: Reporte Listado Viajes Ene-Oct_25.xlsx...
   -> Clustering completado. 2272 rutas clasificadas.
Procesando Financieros (Telemetría + PL)...
   -> Financieros OK. CPK: $41.76, Ociosidad: $3941.96/día

✅ ¡ÉXITO! Archivo 'datos_dashboard_v12.pkl' generado correctamente.
Ahora ejecuta 'app_logistica_final_v12.py'.
